In [1]:
from IPython.display import Image, HTML

# Load and publish CSS
style = HTML(open('talk-IBSim-4i_2021_fvidal.css').read())

display(style)

![IBSim-4i 2021](https://cdn.eventsforce.net/files/ef-q5vmtsq56tk6/website/1653/banner-2.jpg)

# Optimisation and Simulation of X-ray images: Automatic registration of surface models on synchrotron microtomography data

## Franck P. Vidal, Iwan T. Mitchell, and Jean Michel  Létang

<div>
    <img src="A1_FullColour.png" width="150">
    <img src="logo-version1.jpg" width="150">
</div>

# Acknowledgements

- **Jean-Yves Buffière** and **Ce Xiao** of MATEIS laboratory (Lyon), and **Wolfgang Ludwig** of ESRF (Grenoble) for the new projection and CT data,
- **NVIDIA Corporation** for the donation of the NVIDIA TITAN Xp GPU used in the development and validation of gVirtualXRay, 
- **Supercomputing Wales** for the use of its supercomputer, and
- **All the volunteers** who participated to the user study.

# Motivations

- The presence of **strong imaging artefacts** in **microtomographic X-ray** data makes the **CAD modelling process difficult** to carry out. 
- As an **alternative to traditional image segmentation techniques**, we propose to **register the CAD models** by deploying a realistic **X-ray simulation on GPU** in an **optimisation** framework. 
- A **user study** was also conducted to **compare the **measurements made manually** by a cohort of volunteers and **those produced with our framework**. 

# In a Nutshell

- Our implementation relies on open source software only. 
- We numerically modelled the real experiment, taking into account geometrical properties as well as beam hardening, impulse response of the detector, phase contrast, and photon noise. 
- Parameters of the overall model are then optimised so that X-ray projections of the registered the CAD models match the projections from an actual experiment. 

# Context: Artefacts in CT

![Typical example of a reconstructed slice from a fibre composite where strong CT artefacts are present](figure_01.svg)

## Two kinds of artefacts are clearly visible

- Black and white fringes at the silicon carbide edges: these phase artefacts are similar to some image enhancement techniques like unsharp masking and can be modelled by an additive term involving the Laplacian of the phase shift;
 - Dark streaks with bright borders, located along alignments of high density tungsten cores: these artefacts are due to beam hardening and the impulse response of the detector. As voxel values in these streaks are darker than what they should be, it means that the linear attenuation coefficients are underestimated in the artefact areas. 

# Linear attenuation coefficients (&mu;) in cm<sup>-1</sup>

## from the literature (Theoretical), and from the CT slice of the experiment at ESRF (Experimental)

| Structure | Material     | Theoretical | Experimental |
|-----------------|------------------|--------------------|----------------|
| Core         |  W               |   341.61       | 162.34&plusmn;21.67 |
| Fibre         |  SiC            |   2.74            | 5.61&plusmn;5.73  |
| Matrix      | Ti90Al6V4 |   13.13          | 12.87&plusmn;3.57 |

- **W** &mu; are underestimated by a factor of **2**
- **SiC** coefficients are overestimated by a factor of **2**

# Image registration as an optimisation algorithm

![](figure_03.svg)

1. CAD models of the scanned object are generated
2. A CT acquisition is simulated to create X-ray projections from the CAD models
3. Simulated  X-ray projections are compared with the projections from the real experiments
4. An optimisation algorithm tweaks the parameters of the simulation models (CAD & CT acquisition) until convergence

**We need a quick way to simulate realistic X-ray projections.** 

![](https://ibsim.co.uk/events/wp-content/uploads/2020/09/Slide12-1.png)

# Forward Modelling

A projection is often modelled using the polychromatic version of the Beer-Lambert law:
\begin{equation}
    \boldsymbol{I}(x,y) = \sum_i \boldsymbol{R}_i \, \boldsymbol{N}_i \; \exp\left({-\sum_j \mu_j(E_i) \; \boldsymbol{d}_j(x,y)}\right)
    \label{eq:beer}
\end{equation}

When the primary spectrum is not monochromatic the measurement in the equation above, $\boldsymbol{I}$ is the sum of several attenuation laws. We could however compute the effective monochromatic attenuation that would give the same measurement:
\begin{equation}
    \boldsymbol{I}(x,y) = \boldsymbol{I}_0(x,y) \; \exp\left({-\sum_j \mu_j(E_{\mathrm{eff}}) \; \boldsymbol{d}_j(x,y)}\right)
    \label{eq:beer2}
\end{equation}
which is the model considered by CT reconstruction algorithms. **1st source of issue in CT reconstructed images: beam hardening**

# Phase Contrast

The contribution to phase contrast for each material is accumulated in a new image:
\begin{equation}
    \mathbf{Phase} = \sum_j k_j \; \boldsymbol{d}_j * \mathbf{Laplacian}_{\sigma(j)}
    \label{eq:phase}
\end{equation}
<!-- where $k_j$ is a weighting factor corresponding to the contribution of the $j$-th material to phase contrast, 
$\sigma(j)$ a parameter controlling the width of the Laplacian operator, and 
$*$ denotes the convolution of $\boldsymbol{d}_j$ (the path-length image of the $j$-th material) with a Laplacian kernel implemented as follows:
 -->
 
\begin{equation}
    \mathbf{Laplacian}_\sigma(x,y) = \left(\frac{x^2+y^2}{\sigma^4} - \frac{1}{\sigma^2}\right) \exp\left(-\frac{x^2+y^2}{2\sigma^2}\right)
    \label{eq:Laplacian1}
\end{equation}
<!-- The final phase contrast model includes two real numbers for each material $j$, $k_j$ and $\sigma(j)$.
 -->
 
A pixel-wise subtraction `adds' the phase contrast to the raw projections:
\begin{equation}
    \boldsymbol{I}_p = \boldsymbol{I} - \mathbf{Phase}
    \label{eq:Laplacian2}
\end{equation}


# Impulse response of the detector

The response of the camera (i.e.~the line spread-function (LSF) in our case since the sample is translation invariant) is applied row by row by convolution:
\begin{equation}
    \boldsymbol{I}_{\mathrm{LSF}} = \boldsymbol{I}_p * \mathbf{LSF}
\end{equation}
with $\mathrm{LSF}$ modelled as:
\begin{equation}
    \mathbf{LSF} = \left(\frac{a}{c} \left(1+\frac{x^2}{c^2}\right)^{-1}\!\!\!+ \frac{2b}{\sqrt{\pi}d} \exp\left(-\frac{x^2}{d^2}\right) \right) \frac{1}{{\pi}{a}+{2}{b}}
    \label{eq:lsf}
\end{equation}

# Flat-field Correction

Projections are then corrected to account for variations in beam homogeneity and in the pixel-to-pixel sensitivity of the detector:
\begin{equation}
\mathbf{Proj} = \frac{\boldsymbol{I}_{\mathrm{LSF}} - \boldsymbol{D}}{\boldsymbol{F} - \boldsymbol{D}}
\label{eq:flat-fields}
\end{equation}

![](figure_02a.svg)


# Photon noise

Poisson noise is added to the projections  (Lines~67 to 72 of Step~49):
\begin{equation}
\mathbf{NoiseMap} =  \text{Poisson}\left[(\mathbf{Proj} + \text{bias}) \times \text{gain}\right]
\end{equation}
and
\begin{equation}
\mathbf{Proj}_n = \mathbf{Proj} + \text{intensity} \times \mathbf{NoiseMap}
\end{equation}


# Sinogram

We are now able to linearise the transmission tomography data, namely $\mathbf{Proj}_n$ in the previous equation, and we get the sinogram:
\begin{equation}
\textbf{Sino}=-\ln\left(\textbf{Proj}_n\right)
\label{eq:sinogram}
\end{equation}

![](figure_02b.svg)

# CT reconstruction

$\textbf{Sino}$ in the previous equation is the input data used by CT reconstruction algorithms.

![](figure_01.svg)

# Extract the W cores

## With the Hough Transform
![](figure_06.svg)

## With the Otsu method
![](figure_07.svg)

# Experimental set up at ESRF's ID19 - Microtomography beamline

![](figure_05.svg)

# Summary of the parameters that need to be optimised.

|   | **Parameters** |
|---|------------------|
|         1    | Position of the matrix along the primary axis (in {\textmu}m)     |
|         2    | Position of the matrix along the secondary axis (in {\textmu}m)   |
|         3    | Size of the matrix along the primary axis (in {\textmu}m)      |
|         4    | Size of the matrix along the secondary axis (in {\textmu}m)      |
|         5    | Rotation angle of the matrix  (in degrees)                    |
|         6    | Radius of the cores  (in {\textmu}m)                          |
|         7    | Radius of the fibres  (in {\textmu}m)                          |
|         8    | Percentage of 33~keV photons in the beam spectrum               |
|         9    | Percentage of 66~keV photons in the beam spectrum        |
|        10    | Percentage of 99~keV photons in the beam spectrum         |
|        11    | Bias controlling the Poisson noise                        |
|        12    | Gain controlling the Poisson noise                     |
|        13    | Intensity of the Poisson noise                                 |
|        14    | Intensity of the phase contrast for the tungsten core      |
|        15    | Spread of the phase contrast for the tungsten core        |
|        16    | Intensity of the phase contrast for the SiC fibres             |
|        17    | Spread of the phase contrast for the SiC fibres               |
|        18    | Intensity of the phase contrast for the Ti90Al6V4 matrix     |
|        19    | Spread of the phase contrast for the Ti90Al6V4 matrix      |
|        20-23 | Parameters of the LSF |


# Registration pipeline: Divide and Conquert

![](graphical_abstract.png)

# Conclusion & Future Work

- Demonstrated how to automatically produce CAD models as an optimisation problem, producing a high cross-correlation between the experimental CT slice and the simulated CT slice. 
- Choice of objective function is crucial to produce high fidelity results.
- Manual measurements can be variable and subject to bias whereas our framework produced more reliable results. 
- Features seen in the real CT image, including artefacts, were accurately replicated in the CT image reconstructed from the simulated data after registration: 
    - Linear attenuation coefficients are comparable for all the materials, 
    - Geometrical properties are accurately recovered, and 
    - Simulated images reproduce observed experimental artefacts. 

- Fast and realistic simulation of the imaging chain opens up new perspective:
    - For accurate CAD modelling in tomographic X-ray data.
    - Inclusion of such models in iterative reconstruction algorithms.
    - Correct imaging artefacts and improve the value of linear attenuation coefficients. 
